In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from collections import deque
import heapq
import time

class MazePathfinder:
    def __init__(self, root):
        self.root = root
        self.root.title("Maze Pathfinding Visualizer - AI Project")
        
        # Grid settings
        self.rows = 25
        self.cols = 40
        self.cell_size = 20
        
        # Colors
        self.colors = {
            'empty': 'white',
            'wall': 'black',
            'start': 'green',
            'goal': 'red',
            'visited': 'lightblue',
            'path': 'yellow',
            'current': 'orange'
        }
        
        # Grid state
        self.grid = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        self.start = None
        self.goal = None
        self.mode = 'wall'  # wall, start, goal, erase
        self.is_drawing = False
        self.is_running = False
        
        # Statistics
        self.nodes_explored = +0
        self.path_cost = 0
        
        self.setup_ui()
        
    def setup_ui(self):
        # Control panel
        control_frame = ttk.Frame(self.root, padding="10")
        control_frame.pack(side=tk.TOP, fill=tk.X)
        
        # Mode selection
        ttk.Label(control_frame, text="Mode:").pack(side=tk.LEFT, padx=5)
        
        self.mode_var = tk.StringVar(value='wall')
        modes = [('Wall', 'wall'), ('Start', 'start'), ('Goal', 'goal'), ('Erase', 'erase')]
        for text, mode in modes:
            ttk.Radiobutton(control_frame, text=text, variable=self.mode_var, 
                           value=mode, command=self.change_mode).pack(side=tk.LEFT, padx=2)
        
        ttk.Separator(control_frame, orient=tk.VERTICAL).pack(side=tk.LEFT, fill=tk.Y, padx=10)
        
        # Algorithm selection
        ttk.Label(control_frame, text="Algorithm:").pack(side=tk.LEFT, padx=5)
        self.algo_var = tk.StringVar(value='BFS')
        algo_combo = ttk.Combobox(control_frame, textvariable=self.algo_var, 
                                  values=['BFS', 'DFS', 'A*', 'Best-First', 'IDA*'], 
                                  state='readonly', width=12)
        algo_combo.pack(side=tk.LEFT, padx=5)
        
        # Buttons
        ttk.Button(control_frame, text="Find Path", command=self.start_pathfinding).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="Clear Path", command=self.clear_path).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="Clear All", command=self.clear_all).pack(side=tk.LEFT, padx=5)
        
        # Speed control
        ttk.Label(control_frame, text="Speed:").pack(side=tk.LEFT, padx=5)
        self.speed_var = tk.IntVar(value=50)
        speed_scale = ttk.Scale(control_frame, from_=1, to=100, variable=self.speed_var, 
                               orient=tk.HORIZONTAL, length=100)
        speed_scale.pack(side=tk.LEFT, padx=5)
        
        # Info frame for statistics
        info_frame = ttk.Frame(self.root, padding="5")
        info_frame.pack(side=tk.TOP, fill=tk.X)
        
        # Info label
        self.info_label = ttk.Label(info_frame, text="Draw walls, set start (green) and goal (red)", 
                                    font=('Arial', 10))
        self.info_label.pack(side=tk.LEFT, padx=10)
        
        # Statistics labels
        stats_frame = ttk.Frame(info_frame)
        stats_frame.pack(side=tk.RIGHT, padx=10)
        
        self.stats_label = ttk.Label(stats_frame, text="", font=('Arial', 10, 'bold'), 
                                     foreground='blue')
        self.stats_label.pack()
        
        # Canvas for grid
        self.canvas = tk.Canvas(self.root, width=self.cols*self.cell_size, 
                               height=self.rows*self.cell_size, bg='white')
        self.canvas.pack(padx=10, pady=10)
        
        # Bind mouse events
        self.canvas.bind('<Button-1>', self.on_click)
        self.canvas.bind('<B1-Motion>', self.on_drag)
        self.canvas.bind('<ButtonRelease-1>', self.on_release)
        
        self.draw_grid()
        
    def draw_grid(self):
        self.canvas.delete('all')
        for i in range(self.rows):
            for j in range(self.cols):
                x1, y1 = j * self.cell_size, i * self.cell_size
                x2, y2 = x1 + self.cell_size, y1 + self.cell_size
                
                color = self.colors['empty']
                if self.grid[i][j] == 1:
                    color = self.colors['wall']
                elif (i, j) == self.start:
                    color = self.colors['start']
                elif (i, j) == self.goal:
                    color = self.colors['goal']
                elif self.grid[i][j] == 2:
                    color = self.colors['visited']
                elif self.grid[i][j] == 3:
                    color = self.colors['path']
                elif self.grid[i][j] == 4:
                    color = self.colors['current']
                
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline='gray')
        
    def change_mode(self):
        self.mode = self.mode_var.get()
        
    def on_click(self, event):
        if self.is_running:
            return
        self.is_drawing = True
        self.handle_cell(event)
        
    def on_drag(self, event):
        if self.is_drawing and not self.is_running:
            self.handle_cell(event)
            
    def on_release(self, event):
        self.is_drawing = False
        
    def handle_cell(self, event):
        col = event.x // self.cell_size
        row = event.y // self.cell_size
        
        if 0 <= row < self.rows and 0 <= col < self.cols:
            if self.mode == 'wall':
                if (row, col) != self.start and (row, col) != self.goal:
                    self.grid[row][col] = 1
            elif self.mode == 'erase':
                if (row, col) == self.start:
                    self.start = None
                elif (row, col) == self.goal:
                    self.goal = None
                self.grid[row][col] = 0
            elif self.mode == 'start':
                if self.start:
                    old_r, old_c = self.start
                    self.grid[old_r][old_c] = 0
                self.start = (row, col)
                self.grid[row][col] = 0
            elif self.mode == 'goal':
                if self.goal:
                    old_r, old_c = self.goal
                    self.grid[old_r][old_c] = 0
                self.goal = (row, col)
                self.grid[row][col] = 0
            
            self.draw_grid()
    
    def get_neighbors(self, pos):
        r, c = pos
        neighbors = []
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr][nc] != 1:
                neighbors.append((nr, nc))
        return neighbors
    
    def heuristic(self, pos):
        # Manhattan distance heuristic for A*, Best-First, and IDA*
        return abs(pos[0] - self.goal[0]) + abs(pos[1] - self.goal[1])
    
    def visualize_step(self, pos, cell_type='visited'):
        if pos != self.start and pos != self.goal:
            self.grid[pos[0]][pos[1]] = 2 if cell_type == 'visited' else 4
            self.draw_grid()
            delay = (101 - self.speed_var.get()) / 1000
            self.root.update()
            time.sleep(delay)
    
    def reconstruct_path(self, came_from, current):
        path = []
        cost = 0
        while current in came_from:
            current = came_from[current]
            if current != self.start:
                path.append(current)
            cost += 1
        
        for pos in reversed(path):
            self.grid[pos[0]][pos[1]] = 3
            self.draw_grid()
            self.root.update()
            time.sleep(0.05)
        
        # Path cost is the number of steps
        return len(path) + 1, cost + 1
    
    def bfs(self):
        """Breadth-First Search Algorithm - Guarantees Optimal Path"""
        queue = deque([self.start])
        visited = {self.start}
        came_from = {}
        self.nodes_explored = 0
        
        while queue:
            current = queue.popleft()
            self.nodes_explored += 1
            
            if current == self.goal:
                path_len, cost = self.reconstruct_path(came_from, current)
                return True, path_len, cost
            
            self.visualize_step(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    queue.append(neighbor)
        
        return False, 0, 0
    
    def dfs(self):
        """Depth-First Search Algorithm - Does NOT Guarantee Optimal Path"""
        stack = [self.start]
        visited = {self.start}
        came_from = {}
        self.nodes_explored = 0
        
        while stack:
            current = stack.pop()
            self.nodes_explored += 1
            
            if current == self.goal:
                path_len, cost = self.reconstruct_path(came_from, current)
                return True, path_len, cost
            
            self.visualize_step(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    stack.append(neighbor)
        
        return False, 0, 0
    
    def astar(self):
        """A* Search Algorithm - Guarantees Optimal Path with Heuristic"""
        open_set = [(0, self.start)]
        came_from = {}
        g_score = {self.start: 0}
        self.nodes_explored = 0
        
        while open_set:
            _, current = heapq.heappop(open_set)
            self.nodes_explored += 1
            
            if current == self.goal:
                path_len, cost = self.reconstruct_path(came_from, current)
                return True, path_len, cost
            
            self.visualize_step(current)
            
            for neighbor in self.get_neighbors(current):
                tentative_g = g_score[current] + 1
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + self.heuristic(neighbor)
                    heapq.heappush(open_set, (f_score, neighbor))
        
        return False, 0, 0
    
    def best_first_search(self):
        """Best-First Search (Greedy) - Uses Only Heuristic, NOT Optimal"""
        open_set = [(self.heuristic(self.start), self.start)]
        came_from = {}
        visited = {self.start}
        self.nodes_explored = 0
        
        while open_set:
            _, current = heapq.heappop(open_set)
            self.nodes_explored += 1
            
            if current == self.goal:
                path_len, cost = self.reconstruct_path(came_from, current)
                return True, path_len, cost
            
            self.visualize_step(current)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    came_from[neighbor] = current
                    h_score = self.heuristic(neighbor)
                    heapq.heappush(open_set, (h_score, neighbor))
        
        return False, 0, 0
    
    def ida_star_search(self, path, g, bound):
        """Recursive IDA* search function"""
        current = path[-1]
        f = g + self.heuristic(current)
        
        if f > bound:
            return f, None, 0
        
        if current == self.goal:
            return -1, path, g
        
        self.visualize_step(current, 'current')
        self.nodes_explored += 1
        
        min_bound = float('inf')
        for neighbor in self.get_neighbors(current):
            if neighbor not in path:
                path.append(neighbor)
                t, result, cost = self.ida_star_search(path, g + 1, bound)
                
                if t == -1:
                    return -1, result, cost
                if t < min_bound:
                    min_bound = t
                
                path.pop()
        
        return min_bound, None, 0
    
    def ida_star(self):
        """Iterative Deepening A* Algorithm - Optimal and Memory Efficient"""
        bound = self.heuristic(self.start)
        path = [self.start]
        self.nodes_explored = 0
        
        while True:
            t, result, cost = self.ida_star_search(path, 0, bound)
            
            if t == -1:
                for pos in result[1:-1]:
                    self.grid[pos[0]][pos[1]] = 3
                self.draw_grid()
                return True, len(result), cost
            
            if t == float('inf'):
                return False, 0, 0
            
            bound = t
    
    def start_pathfinding(self):
        if self.is_running:
            return
        
        if not self.start or not self.goal:
            messagebox.showwarning("Warning", "Please set both start and goal positions!")
            return
        
        self.is_running = True
        self.clear_path()
        
        algo = self.algo_var.get()
        self.info_label.config(text=f"Running {algo}...")
        self.stats_label.config(text="")
        self.root.update()
        
        start_time = time.time()
        
        if algo == 'BFS':
            found, path_len, cost = self.bfs()
        elif algo == 'DFS':
            found, path_len, cost = self.dfs()
        elif algo == 'A*':
            found, path_len, cost = self.astar()
        elif algo == 'Best-First':
            found, path_len, cost = self.best_first_search()
        else:  # IDA*
            found, path_len, cost = self.ida_star()
        
        elapsed = time.time() - start_time
        
        if found:
            # Determine if path is optimal
            optimal_algos = ['BFS', 'A*', 'IDA*']
            is_optimal = algo in optimal_algos
            optimal_text = " (OPTIMAL)" if is_optimal else " (NOT OPTIMAL)"
            
            self.info_label.config(
                text=f"{algo} completed in {elapsed:.3f}s | Nodes explored: {self.nodes_explored}"
            )
            self.stats_label.config(
                text=f"✓ Path Found! | Path Length: {path_len} | Cost: {cost}{optimal_text}"
            )
            
            # Print to console
            print(f"\n{'='*60}")
            print(f"Algorithm: {algo}")
            print(f"Result: PATH FOUND")
            print(f"Path Length: {path_len} cells")
            print(f"Optimal Path Cost: {cost} steps")
            print(f"Nodes Explored: {self.nodes_explored}")
            print(f"Time Taken: {elapsed:.3f} seconds")
            print(f"Is Optimal: {'YES' if is_optimal else 'NO'}")
            print(f"{'='*60}\n")
            
        else:
            self.info_label.config(
                text=f"{algo}: No path found in {elapsed:.3f}s | Nodes explored: {self.nodes_explored}"
            )
            self.stats_label.config(text="✗ No Path Found!")
            messagebox.showinfo("Result", "No path found!")
            
            # Print to console
            print(f"\n{'='*60}")
            print(f"Algorithm: {algo}")
            print(f"Result: NO PATH FOUND")
            print(f"Nodes Explored: {self.nodes_explored}")
            print(f"Time Taken: {elapsed:.3f} seconds")
            print(f"{'='*60}\n")
        
        self.is_running = False
    
    def clear_path(self):
        """Clear the visualization (visited cells and path)"""
        for i in range(self.rows):
            for j in range(self.cols):
                if self.grid[i][j] in [2, 3, 4]:
                    self.grid[i][j] = 0
        self.draw_grid()
        self.stats_label.config(text="")
    
    def clear_all(self):
        """Clear everything including walls, start, and goal"""
        self.grid = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        self.start = None
        self.goal = None
        self.draw_grid()
        self.info_label.config(text="All cleared - Draw walls, set start and goal")
        self.stats_label.config(text="")

# Run the application
def run_app():
    root = tk.Tk()
    app = MazePathfinder(root)
    root.mainloop()

# Execute this to start the visualizer
if __name__ == "__main__":
    print("🚀 Starting Maze Pathfinding Visualizer...")
    print("\n📋 Instructions:")
    print("1. Draw walls by clicking and dragging (black)")
    print("2. Select 'Start' mode and click to place green start point")
    print("3. Select 'Goal' mode and click to place red goal point")
    print("4. Choose an algorithm: BFS, DFS, A*, Best-First, or IDA*")
    print("5. Click 'Find Path' to watch the algorithm work!")
    print("\n🎨 Color Legend:")
    print("- Green: Start position")
    print("- Red: Goal position")
    print("- Black: Walls/Obstacles")
    print("- Light Blue: Visited cells")
    print("- Yellow: Final path")
    print("- Orange: Current cell (IDA* only)")
    print("\n🔍 Algorithm Comparison:")
    print("- BFS: Guarantees optimal path, explores uniformly")
    print("- DFS: Memory efficient, does NOT guarantee optimal path")
    print("- A*: Optimal path using heuristic (f = g + h)")
    print("- Best-First: Greedy search using only heuristic, NOT optimal")
    print("- IDA*: Memory efficient optimal search")
    print("\n📊 Statistics displayed:")
    print("- Path Length: Number of cells in the path")
    print("- Optimal Path Cost: Total steps from start to goal")
    print("- Nodes Explored: Number of cells examined")
    print("- Time Taken: Execution time in seconds")
    print("\n" + "="*60 + "\n")
    
    run_app()

🚀 Starting Maze Pathfinding Visualizer...

📋 Instructions:
1. Draw walls by clicking and dragging (black)
2. Select 'Start' mode and click to place green start point
3. Select 'Goal' mode and click to place red goal point
4. Choose an algorithm: BFS, DFS, A*, Best-First, or IDA*
5. Click 'Find Path' to watch the algorithm work!

🎨 Color Legend:
- Green: Start position
- Red: Goal position
- Black: Walls/Obstacles
- Light Blue: Visited cells
- Yellow: Final path
- Orange: Current cell (IDA* only)

🔍 Algorithm Comparison:
- BFS: Guarantees optimal path, explores uniformly
- DFS: Memory efficient, does NOT guarantee optimal path
- A*: Optimal path using heuristic (f = g + h)
- Best-First: Greedy search using only heuristic, NOT optimal
- IDA*: Memory efficient optimal search

📊 Statistics displayed:
- Path Length: Number of cells in the path
- Optimal Path Cost: Total steps from start to goal
- Nodes Explored: Number of cells examined
- Time Taken: Execution time in seconds



Algorithm: 